In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pickle
import sys
import time
import numpy as np

In [ ]:
sys.setrecursionlimit(1000000)

In [2]:
# function to extract all article links from the mewspaper zackzack
def spider(year, month):
    article_list = []
    
    page = 1
    source_x = requests.get(f'https://zackzack.at/{year}/{month}/page/{page}/')
    
    
    while source_x.status_code == 200:
        # year and month as break points
        if year == 2021 and month == 6:
            break
        source = requests.get(f'https://zackzack.at/{year}/{month}/page/{page}/').text
        source_x = requests.get(f'https://zackzack.at/{year}/{month}/page/{page}/')
        soup = BeautifulSoup(source, 'lxml')

        for link in soup.find_all('h3', class_='slide-entry-title entry-title'):
            article_link = link.a.get('href')
            article_list.append(article_link)
        page += 1    
        
        
        if source_x.status_code != 200:
            page = 1
            month += 1
            source = requests.get(f'https://zackzack.at/{year}/{month}/page/{page}/').text
            soup = BeautifulSoup(source, 'lxml')

            for link in soup.find_all('h3', class_='slide-entry-title entry-title'):
                article_link = link.a.get('href')
                
                article_list.append(article_link)
            page += 1
            source_x = requests.get(f'https://zackzack.at/{year}/{month}/page/{page}/')
           
        
        elif month > 12:
            page = 1
            month = 1
            year += 1
            source = requests.get(f'https://zackzack.at/{year}/{month}/page/{page}/').text
            soup = BeautifulSoup(source, 'lxml')

            for link in soup.find_all('h3', class_='slide-entry-title entry-title'):
                article_link = link.a.get('href')
                article_list.append(article_link)
            
            page += 1
            source_x = requests.get(f'https://zackzack.at/{year}/{month}/page/{page}/')
         #create a pickle file of links
    #!mkdir pickle_files
    pickle_out = open('article_linklist', 'wb')
    pickle.dump(article_list, pickle_out)
    pickle_out.close()
    
    #print(article_linklist)
    return 'Finished'

In [3]:
spider(2021, 5)

'Finished'

In [10]:
pickle_in = open('files/article_linklist', 'rb')
article_linklist = pickle.load(pickle_in)

In [13]:
len(article_linklist)

5582

In [ ]:
# storing all the soup data of the article links in a variable
data = []
idx = 0
pause = 10
while idx < len(article_linklist):
    try:
        source = requests.get(article_linklist[idx]).text
        soup = BeautifulSoup(source, 'lxml')
        data.append(soup)
        idx += 1
    except:
        time.sleep(pause)
        pause += 10

In [ ]:
# saving the soup data to a txt-file
souplist_prettified = [k.prettify() for k in data]
souplist_withword = [m + 'BREAKHERE' for m in souplist_prettified]
souplist_as_one = ''.join(souplist_withword)
with open('souplist.txt', 'w', encoding='utf-8') as file:
    file.write(souplist_as_one)